# BeverageCo: Multi-period Production Planning

## Problem Background
BeverageCo produces three types of beverages: Premium Soda, Energy Drink, and Sparkling Water. They need to plan production over a 6-month horizon considering:
- Seasonal demand patterns
- Production line setup costs and times
- Limited shelf life for products
- Storage capacity constraints
- Workforce scheduling

## Business Context
- Premium Soda has highest profit margin but shortest shelf life (3 months)
- Energy Drink has medium margin and 4-month shelf life
- Sparkling Water has lowest margin but 6-month shelf life
- Storage costs increase in summer months (refrigeration)
- Demand peaks during summer (months 3-4)

## Optimization Goals
1. Maximize total profit
2. Meet demand without stockouts
3. Minimize waste from expired products
4. Balance production line utilization
5. Optimize workforce scheduling

In [ ]:
import pulp
import pandas as pd
import numpy as np

# Problem parameters
products = ['Premium', 'Energy', 'Sparkling']
months = range(1, 7)  # 6-month planning horizon
shelf_life = {'Premium': 3, 'Energy': 4, 'Sparkling': 6}

# Production costs and capacities
production_cost = {
    'Premium': 2.5,
    'Energy': 1.8,
    'Sparkling': 1.2
}

setup_cost = {
    'Premium': 500,
    'Energy': 400,
    'Sparkling': 300
}

# Storage costs (higher in summer months 3-4)
storage_cost = {
    'Premium': {1: 0.3, 2: 0.3, 3: 0.5, 4: 0.5, 5: 0.3, 6: 0.3},
    'Energy': {1: 0.2, 2: 0.2, 3: 0.4, 4: 0.4, 5: 0.2, 6: 0.2},
    'Sparkling': {1: 0.1, 2: 0.1, 3: 0.3, 4: 0.3, 5: 0.1, 6: 0.1}
}

# Selling prices
price = {
    'Premium': 5.0,
    'Energy': 4.0,
    'Sparkling': 3.0
}

# Monthly demand (higher in summer)
demand = {
    'Premium': {1: 1000, 2: 1200, 3: 2000, 4: 1800, 5: 1100, 6: 900},
    'Energy': {1: 800, 2: 1000, 3: 1500, 4: 1400, 5: 900, 6: 700},
    'Sparkling': {1: 1200, 2: 1400, 3: 2200, 4: 2000, 5: 1300, 6: 1100}
}

# Create optimization model
prob = pulp.LpProblem("BeverageCo_Production_Planning", pulp.LpMaximize)

# Decision variables
production = pulp.LpVariable.dicts("production",
    ((p, m) for p in products for m in months),
    lowBound=0)

inventory = pulp.LpVariable.dicts("inventory",
    ((p, m) for p in products for m in months),
    lowBound=0)

setup = pulp.LpVariable.dicts("setup",
    ((p, m) for p in products for m in months),
    cat='Binary')

# Objective function: Maximize profit
prob += (
    # Revenue
    pulp.lpSum(price[p] * demand[p][m] for p in products for m in months) -
    # Production costs
    pulp.lpSum(production_cost[p] * production[p,m] for p in products for m in months) -
    # Setup costs
    pulp.lpSum(setup_cost[p] * setup[p,m] for p in products for m in months) -
    # Storage costs
    pulp.lpSum(storage_cost[p][m] * inventory[p,m] for p in products for m in months)
)

# Add constraints here...
# 1. Inventory balance constraints
# 2. Shelf life constraints
# 3. Storage capacity constraints
# 4. Production capacity constraints
# 5. Setup forcing constraints

# Solve the model
prob.solve()

print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Optimal Profit: ${pulp.value(prob.objective):,.2f}")